In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import sys
sys.path.append('/home/baraa/.local/lib/python3.10/site-packages')

In [3]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.1.2 (SDL 2.0.16, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model =load_model('/home/baraa/Documents/FYP/Models')

In [6]:

mixer.init()

def get_prediction(frame):
    try:
        closed = 0
        opened = 0
        
        height,width = frame.shape[0:2]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
        eyes = eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)

        cv2.rectangle(frame, (0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)


        for (ex,ey,ew,eh) in eyes:
            # preprocessing steps
            eye= frame[ey:ey+eh,ex:ex+ew]
            eye= cv2.resize(eye,(80,80))
            eye= eye/255
            eye= eye.reshape(80,80,3)
            eye= np.expand_dims(eye,axis=0)
            # preprocessing is done now model prediction
            prediction = model.predict(eye)

            # if eyes are closed
            if prediction[0][0]>prediction[0][1]:
                closed += 1

            # if eyes are open
            elif prediction[0][1]>prediction[0][0]:
                opened += 1
        return opened, closed
    
    except:
        return "invalid image type"

    


for i in range(1, 5):
    frame = cv2.imread(f'/home/baraa/Pictures/p{i}.jpg')
    print (f'/home/baraa/Pictures/p{i}.jpg')
    opened, closed = get_prediction(frame)
    if (opened > closed):
        print ("Open")
    else:
        print ("Closed")


/home/baraa/Pictures/p1.jpg
1/1 [==============================] - 0s 31ms/step
Closed
/home/baraa/Pictures/p2.jpg
1/1 [==============================] - 0s 32ms/step
Closed
/home/baraa/Pictures/p3.jpg
1/1 [==============================] - 0s 33ms/step
Closed
/home/baraa/Pictures/p4.jpg
1/1 [==============================] - 0s 31ms/step
Closed
